<center><h1><b>Proposed changes in the architecture with focus on overhauling import functionality </b></h1></center>


<img src ="importchangeshld.png" />

The above design is as per the CQRS philosophy. Chnage is sought primarily with respect to the "C-Command" of the CQRS with respect to Add/Update/Delete operations to primarily enable a cleaner seaparation of the logic across different application layers/tiers. The "Q-Query" of the CQRS is best left to retain the existing logic of receiveing data from the BAService or SOADataGridService till the time the frontend application is not ready to be changed.

<h2><b>Detailed explanation</b></h2>

Tracing back to history Emisoft platform is based on a solution which is modelled according to an industry domain wherein the solution depicts the entities and the relationships as pervalent in the domain and this is primarily modelled by the configurators using the solution designer. When it comes to designing the runtime for the end users based on this solution during the software development stages effectively it tended to fit the domain driven design philosphy whereby if the domain entities with their respective behavior and validations self contained  could be generated based on how the solution configurators defined it then it should be very easy to design the application in true DDD philosphy. 
In the existing design the DDD tenet is partially done due to the true types of the entities not being run at dynamic and over the yers much thought has been gone to how to utilize the true types of the entities by dint of which the application could be made much simpler and also performance efficient. Also it needs to be borne in mind that during the initial conception stage the idea was way ahead of the technology but now new technology trends are emerging in the Microsoft space itself with .Net Core whereby doamin entiy types as in the true sense modelled by the configurators acn be utilized as is by the platform runtime.
Hereby we are presented a chnace to capitalize on the technology trend and move on to the .Net Core platform to lay the foundation for future complete migration by the overhaul required of impot functionality.
Note of caution: While moving to .Net Core the tendency would be to move everything related to .Net Core, this is is really unnecessary as it is worth remembering that Microsoft is slated to release .Net 5 in November 2020 which is a unified framework combing both he the .Net framework and the .Net Core worlds.  

<h2><b>Technical details</b></h2>

The different players involved as is being conceptualized.

<h3><b>Domain entity(ies)</b></h3>
    

The generated server model containing the domain entities along with the different methods as designed by the solution configurators will be the domain entities coming into play. As we see withut much chnage it can be made comaptible to .Net standard and made use of in .Net Core as is. For transfer over the wire from UI to the web server whether DTO needs to be modelled on this domain entity is a personal choice and DTOs can be generated if deemed necessary. But since now over web api we use json to map an entity so we donot necessarily need DTO. AS we are not modifying the query operations so all the query logic as existing written in the methods of this entities can be left as is. Also it can be made more maningful if with attributes relevant validations are made inbuilt.

<h3><b>Data web API</b></h3>

This is the proposed new web API that will expose methods primarily to conform to POT, PUT, DELETE verbs as prevalent in REST terminology and which will accept type in the form of json string conforming to domain entity. Any front end or relevant will communicate with this web api as far as CUD operations are concerned.

<h3><b>Domain Service layer (optional)</b></h3>

When using asp.net core and EF Core this layer is a bit meaningless as EF Core DbContext itself acts as the UOW-repository combination and if we have modelled the domain entity to be non-anaemic then the service layer becomes irrelevant. It thus becomes a personal choice to use this.

<h3><b>Domain validator</b></h3>

This can be used to ensure cleaner separation of the validation logic with respect to the domain entities as might be defined by the solution modelled from the main api layer.

<h3><b>Solution DBContext</b></h3>

The backbone when using EF Core. A custom DBContext inherited from the DBContext pertaining to the solution needs to be generated whereby for every domain entity there is a corresponding DBSet defined in this DBContext. This DBContext are what maps the domain entities to the corresponding underlying datbase tables and the dbcontext can also contain the rules related to database validation duplicating the domain validation if need be. The DBContext can define Save, Rollback, Commit methods which are appropriately called by the appropriate methods in the data layer.

<h2><b>Sample Implementation details with respect to the CUD API</b></h2>

If we are able to generate a custyom dbContext conforming and mapping to the solution domain entities like as below:

In [ ]:
public class SolutionDbContext : DbContext 
{
    public SolutionDbContext(DbContextOptions<LasteDbContext> options) 
            : base(options) 
    {
    }
    
 
    public DbSet<EnergyConsumption> EnergyConsumption { get; set; }
    public DbSet<FuelConsumption> FuelConsumption { get; set; }
    
    
    private IDbContextTransaction _transaction;
 
    public void BeginTransaction()
    { 
        _transaction = Database.BeginTransaction();
    }
 
    public void Commit()
    {
        try
        {
            SaveChanges();
            _transaction.Commit();
        }
        finally
        {
            _transaction.Dispose();
        }        
    }
 
    public void Rollback()
    { 
        _transaction.Rollback();
        _transaction.Dispose();
    }
}
Note: FuelConsumption or EnerygyConsimption are domain entities as being utilized from the existing ServerModel

Then we are able to define a data web api or modify the existing data web api created accordingly as below where the essence is being shown:

In [ ]:
[Route("api/[controller]")]
public class BaseController<T> : Controller where T : class
{
    private readonly SolutionDBContext _context;
 
    public BaseController(SolutionDBContext context) => _context = context;
    
     
    [HttpPost]
    public async Task<ActionResult<t>> Post([FromBody]T entity)
    {
            //call DomainValidator.Validate(entity);
            //pre save processing inclusive of OnPosting() and running calculation workflow thru workflow service by raising qan event.
            Guid? id = typeof(T).id;
            if (id == null) 
            {
                _context.Set<TEntity>().Add(entity);
                await _context.SaveChangesAsync();
                //run recalculation thru recalculation service
                return CreatedAtAction(typeof(T).Name, new { id = typeof(T).Id }, entity);
            }
            else
            {
                var editedEntity = _context.Set<TEntity>().FirstOrDefault(e => e.Id == entity.Id);
                editedEntity = entity; 
                editedEnty.State = EntitySate.Modified;
                try
                {
                    await _context.saveChangesAsync();
                    //run recalculation thru recalculation service. calling a service from service is not a rcommended practice so best way is to raise domain
                    //event and transfer to a message bus an execute the service from that message 
                }
                catch
                {
                    throw;
                }
                return NoContent();
                
            }
        
    }
    
    [HttpDelete("{id}")]
    public async Task<ActionResult<T>> Delete(Guid id)
    {
        var entityToDelete = _context.Set<TEntity>().FirstOrDefault(e => e.Id == id);
        //perform any pre operation
        if (entityToDelete == null)
        {
            return NotFound();
        }
         _context.Set<TEntity>().Remove(entityToDelete);
        await _context.SaveChangesAsync();
        return T;    
        
    }
    
}

It would be a worthwhile to try to move behaviors as pertains to doamin entity in the domain entity itself instead of scattering across the application layers. 

There are some logic which are best left not ported at this juncture like recalculation and calculation workflow execution. For example during save this are needed to be called, the best way to call them would be to call them via their respective service(s) vide raising an event, a corresponding event listener will pick up the event and delegate the call to the respective service. 

<h2><b>Conclusion</b></h2>

What are we gaining by the changes? <br/>
(i) The primary goal is to have a cleaner separation across the layers in terms of code so the functionalities are not scattered across the layers. <br/>(ii) As much as possible across all the sectrum of development try to reflect what the configurators intended to model by giving special attention to the domain entities and using them rather than a generic entity structure which can result in gap in usage in terms of configurators and devlopers.<br/> (iii) For the data layer in the runtime solution right now the queries as fired for the CUD operations are written by hand and are more likely to be error prone and also ineffciennt as also with respect to handling of transactions. By going for a tried and test EF Core to handle the CUD operations will at least ensure efficient, well tested queries as well also the entity along with its child entities can be in one go.<br/> 
(iv) Implementing .Net Core gives a little ease of convenience in terms of development at least in terms of injecvting classes and utilizing them, and by its lightweightedness it is guaranteed to be a bit more performance efficient than .net Framework. But as noted with the slated rel;ease of .Net 5 in the near future this distinction between .net core and .net Framework is going to be blurred.   <br/>      
